In [40]:
from astropy.time import Time
import astropy.units as u

import astromet

from random import randint
import matplotlib.pyplot as plt

import scanninglaw.times as times
from scanninglaw.source import Source

import numpy as np
from astropy import units as u

mas = (1.0*u.mas).to(u.deg).value

In [41]:
### use Times New Roman font in Figures ####
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'DeJavu Serif'
plt.rcParams['font.serif'] = ['Times New Roman']

In [42]:
def generate_random_digits(n: int):
    return int(''.join(["{}".format(randint(0, 9)) for num in range(0, n)]))

In [43]:
#create these directories if running for the first time

In [44]:
from scanninglaw.config import config
#change these directories if necessary
config['data_dir'] = '/d7/CAC/tj0014/Projects/GAME/Taj_GAME/scanninglaw'
game_dir = '/d7/CAC/tj0014/Projects/GAME/Taj_GAME/'
sing_source_dir = game_dir+'/sing_source/'
microlensing_dir = game_dir+'/microlensing/'
microlensing_only_dir = game_dir+'/microlensing_only/'
binary_dir = game_dir+'/binary/'
figure_dir = game_dir+'/Figures/'

#if running for the first time, you have to create these directories
#mkdir $game_dir/sing_source/
#mkdir $game_dir/microlensing/
#mkdir $game_dir/binary/
#mkdir $game_dir/Figures/

# Run the first time if you haven't downloaded the times yet
# times.fetch(version='dr3_nominal')

In [45]:
from typing import Dict
import pandas as pd
import numpy as np

def to_mock_lpc(
    source_id: float, mean_g_mag: float, ref_epoch: float,
    gaia_results: Dict[str, float],
    x_obs: np.array, x_obs_no_lensing: np.array,
    x_err: np.array, phi_obs: np.array,
    rac_obs: np.array, dec_obs: np.array,
    nmeasure: int = 9) -> pd.DataFrame:

    n_rows = x_obs.shape[0]

    random_transit_id = generate_random_digits(12)
    random_obmt = generate_random_digits(17)
    n_range = np.repeat(np.arange(int(n_rows/nmeasure)), nmeasure)

    mock_obmt_rev = np.repeat(np.linspace(1200, 9000, int(n_rows/nmeasure)), nmeasure)+\
        np.repeat(np.linspace(0., 0.01, nmeasure).reshape((-1, 1)), int(n_rows/nmeasure), axis=1).T.flatten()

    mock_fov = np.repeat(np.repeat(np.array([0, 1]), nmeasure).reshape((-1, 1)).T,
        int(n_rows/(2*nmeasure))+nmeasure, axis=0).flatten()[:n_rows]

    mock_strip = np.repeat(np.arange(0, 12).reshape((-1, 1)), int(n_rows/nmeasure), axis=1).T.flatten()[:n_rows]
    
    df: pd.DateFrame = pd.DataFrame(
        data = {
            'sourceId': np.repeat(source_id, n_rows),
            'transitId': random_transit_id+n_range,
            'obmt': random_obmt+np.arange(n_rows),
            'w': x_obs,
            'z': x_obs*10,
            'wError': x_err,
            'zError': x_err*10,
            'wDiff': x_obs-x_obs_no_lensing,
            'zDiff': (x_obs-x_obs_no_lensing)*10,
            'theta': phi_obs,
            'varpiFactorAl': np.repeat(0.5, n_rows),
            'varpiFactorAc': np.repeat(1., n_rows),
            'ra': np.repeat(gaia_results['ra_ref'], n_rows),
            'dec': np.repeat(gaia_results['dec_ref'], n_rows),
            'varpi': np.repeat(gaia_results['parallax'], n_rows),
            'varpiError': np.repeat(gaia_results['parallax_error'], n_rows),
            'pmra': np.repeat(gaia_results['pmrac'], n_rows),
            'pmraError': np.repeat(gaia_results['pmrac_error'], n_rows),
            'pmdec': np.repeat(gaia_results['pmdec'], n_rows),
            'pmdecError': np.repeat(gaia_results['pmdec_error'], n_rows),
            'refEpoch': np.repeat(ref_epoch, n_rows),
            'GMag': np.repeat(mean_g_mag, n_rows),
            'nuEff': np.repeat(0.1, n_rows),
            'excessNoise': np.repeat(gaia_results['excess_noise'], n_rows),
            'chi2Al': np.repeat(gaia_results['chi2'], n_rows),
            'chi2Ac': np.repeat(gaia_results['chi2'], n_rows)*10,
            'a': rac_obs,
            'd': dec_obs,
            'elapsedNanoSecs': ((t_obs-2010.)*u.year).to(u.nanosecond).value,
            'fov': mock_fov,
            'row': np.random.choice([4, 6], size=n_rows),
            'strip': mock_strip,
            'mu': np.repeat(0, n_rows),
            'gate': np.repeat(0, n_rows),
            'gclass': np.repeat(1, n_rows),
            'ccdProcFlags': np.repeat(0, n_rows),
            'transitProcFlags': np.repeat(0, n_rows),
            'alrate': np.repeat(np.random.normal(), n_rows),
            'acrate': np.repeat(np.random.normal()*1e-6, n_rows),
            'omega': np.linspace(-3, 3, n_rows),
            'obmt_rev': mock_obmt_rev,
            'subpixel': np.random.uniform(0, 1, size=n_rows)
        }
    )

    return df

In [46]:
###################################################################
### Function that adds ldracs, lddecs, and mag_diff to .parquet #####
###################################################################

def to_mock_lpc_add(
    source_id: float, mean_g_mag: float, ref_epoch: float,
    gaia_results: Dict[str, float],
    x_obs: np.array, x_obs_no_lensing: np.array,
    x_err: np.array, phi_obs: np.array,
    rac_obs: np.array, dec_obs: np.array,
    ldracs: np.array, lddecs: np.array, mag_diff: np.array,
    nmeasure: int = 9) -> pd.DataFrame:

    n_rows = x_obs.shape[0]

    random_transit_id = generate_random_digits(12)
    random_obmt = generate_random_digits(17)
    n_range = np.repeat(np.arange(int(n_rows/nmeasure)), nmeasure)

    mock_obmt_rev = np.repeat(np.linspace(1200, 9000, int(n_rows/nmeasure)), nmeasure)+\
        np.repeat(np.linspace(0., 0.01, nmeasure).reshape((-1, 1)), int(n_rows/nmeasure), axis=1).T.flatten()

    mock_fov = np.repeat(np.repeat(np.array([0, 1]), nmeasure).reshape((-1, 1)).T,
        int(n_rows/(2*nmeasure))+nmeasure, axis=0).flatten()[:n_rows]

    mock_strip = np.repeat(np.arange(0, 12).reshape((-1, 1)), int(n_rows/nmeasure), axis=1).T.flatten()[:n_rows]

    df: pd.DateFrame = pd.DataFrame(
        data = {
            'sourceId': np.repeat(source_id, n_rows),
            'transitId': random_transit_id+n_range,
            'obmt': random_obmt+np.arange(n_rows),
            'w': x_obs,
            'z': x_obs*10,
            'wError': x_err,
            'zError': x_err*10,
            'wDiff': x_obs-x_obs_no_lensing,
            'zDiff': (x_obs-x_obs_no_lensing)*10,
            'theta': phi_obs,
            'varpiFactorAl': np.repeat(0.5, n_rows),
            'varpiFactorAc': np.repeat(1., n_rows),
            'ra': np.repeat(gaia_results['ra_ref'], n_rows),
            'dec': np.repeat(gaia_results['dec_ref'], n_rows),
            'varpi': np.repeat(gaia_results['parallax'], n_rows),
            'varpiError': np.repeat(gaia_results['parallax_error'], n_rows),
            'pmra': np.repeat(gaia_results['pmrac'], n_rows),
            'pmraError': np.repeat(gaia_results['pmrac_error'], n_rows),
            'pmdec': np.repeat(gaia_results['pmdec'], n_rows),
            'pmdecError': np.repeat(gaia_results['pmdec_error'], n_rows),
            'refEpoch': np.repeat(ref_epoch, n_rows),
            'GMag': np.repeat(mean_g_mag, n_rows),
            'nuEff': np.repeat(0.1, n_rows),
            'excessNoise': np.repeat(gaia_results['excess_noise'], n_rows),
            'chi2Al': np.repeat(gaia_results['chi2'], n_rows),
            'chi2Ac': np.repeat(gaia_results['chi2'], n_rows)*10,
            'a': rac_obs,
            'd': dec_obs,
            'elapsedNanoSecs': ((t_obs-2010.)*u.year).to(u.nanosecond).value,
            'fov': mock_fov,
            'row': np.random.choice([4, 6], size=n_rows),
            'strip': mock_strip,
            'mu': np.repeat(0, n_rows),
            'gate': np.repeat(0, n_rows),
            'gclass': np.repeat(1, n_rows),
            'ccdProcFlags': np.repeat(0, n_rows),
            'transitProcFlags': np.repeat(0, n_rows),
            'alrate': np.repeat(np.random.normal(), n_rows),
            'acrate': np.repeat(np.random.normal()*1e-6, n_rows),
            'omega': np.linspace(-3, 3, n_rows),
            'obmt_rev': mock_obmt_rev,
            'subpixel': np.random.uniform(0, 1, size=n_rows),
            'ldracs': ldracs,
            'lddecs': lddecs,
            'mag_diff': mag_diff
        }
    )
    #print(df['ldracs'].values)
    return df

# GAIA scanning law: determine which point (REC, DEC) is visited the mosto often with GAIA 

In [47]:
import astropy.units as units

rec_list_1d = np.linspace(0.0, 360.0, 500)
dec_list_1d = np.linspace(-90.0,90.0, 250)
rec_list, dec_list = np.meshgrid(rec_list_1d, dec_list_1d)

coords = Source(rec_list*units.deg, dec_list*units.deg, frame='icrs')

scl = times.Times(version='dr3_nominal')

scantimes = scl(coords)


Loading auxilliary data ...



KeyboardInterrupt



In [ ]:
#create the figure
fig = plt.figure(figsize=(12,4), dpi=150)

nscan=np.asarray(scantimes['counts'][0], dtype=float) # [0] is for FOV1, [1] is for FOV2: https://scanninglaw.readthedocs.io/en/latest/examples.html
#find max number of GAIA visits
from numpy import unravel_index
i_max = unravel_index(nscan.argmax(), nscan.shape) 

plt.xlabel('rec[deg]')
plt.ylabel('dec[deg]')
# Number of maximum values you want to find
N = 5
# Use argpartition to get the indices of the top N maximum values
indices = np.argpartition(nscan.flatten(), -N)[-N:]
# Convert the flattened indices to row and column indices
row_indices, col_indices = np.unravel_index(indices, nscan.shape)
# Create a list of tuples containing the row and column indices of the top N maximum values
top_N_indices = list(zip(row_indices, col_indices))

im=plt.imshow(nscan,origin='lower',interpolation='nearest',cmap='plasma', aspect='equal',extent=[0,360,-90,90])
plt.scatter([312.29286], [44.54631],c='green')
plt.scatter(rec_list_1d[i_max[1]], dec_list_1d[i_max[0]],c='blue')
plt.scatter(rec_list_1d[col_indices[0]], dec_list_1d[row_indices[0]],c='black')
plt.scatter(rec_list_1d[col_indices[1]], dec_list_1d[row_indices[1]],c='black')
plt.scatter(rec_list_1d[col_indices[2]], dec_list_1d[row_indices[2]],c='black')
plt.scatter(rec_list_1d[col_indices[3]], dec_list_1d[row_indices[3]],c='black')

cbar = fig.colorbar(im,pad=0.01)
cbar.minorticks_on()
cbar.set_label(r'$N$')

plt.savefig('map.png', bbox_inches='tight', dpi=150)
print('Coordinates of max positions (RA,DEC):',rec_list_1d[col_indices[0]], dec_list_1d[row_indices[0]])
print('Coordinates of max positions (RA,DEC):',rec_list_1d[col_indices[1]], dec_list_1d[row_indices[1]])
print('Coordinates of max positions (RA,DEC):',rec_list_1d[col_indices[2]], dec_list_1d[row_indices[2]])
print('Coordinates of max positions (RA,DEC):',rec_list_1d[col_indices[3]], dec_list_1d[row_indices[3]])
print('Coordinates of max positions (RA,DEC):',rec_list_1d[col_indices[4]], dec_list_1d[row_indices[4]])

In [ ]:
dr3_sl = times.Times(version='dr3_nominal')

RA: float = rec_list_1d[i_max[1]]
DEC: float = dec_list_1d[i_max[0]]

#RA: float = 186.85370741482967 
#DEC: float = 47.349397590361434

REFERENCE_EPOCH: float = 2016. # DR3

c = Source(RA, DEC, unit='deg')

sl=dr3_sl(c, return_times=True, return_angles=True)

ts=np.squeeze(np.hstack(sl['times'])).astype('double')
sort=np.argsort(ts)
ts=2010+ts[sort]/365.25
phis=np.squeeze(np.hstack(sl['angles']))[sort].astype('double')
mean_g_mag = 14 # change here - Lukasz recommended between 14-15 (Martina had different values: 15.16, 19,...)

# Generating mock data for the case of microlensing

In [ ]:
cd $microlensing_only_dir

In [ ]:
# check times t0 and tE
Time(7570+2450000., format='jd').decimalyear
#Time(7000+2450000., format='jd').decimalyear
print('in days:',(30*u.day).to(u.year).value)
(ts[np.where(np.abs(ts-2016)<0.5)[0]]-2014)*365

In [ ]:
t0_plot=Time(7765+2450000., format='jd').decimalyear #2015.65
#t0_plot=2016
print(t0_plot)
hist,bins=np.histogram(ts,bins=100)
print(hist)
dtE_min,dtE_max=70/365,180/365
plt.hist(ts,bins=100)
plt.vlines(t0_plot,ymin=0,ymax=60,color='red',label='t0')
plt.vlines([t0_plot-dtE_min,t0_plot+dtE_min],ymin=0,ymax=60,color='purple',linestyle='dashed',label='tE_lower_limit')
plt.vlines([t0_plot-dtE_max,t0_plot+dtE_max],ymin=0,ymax=60,color='orange',linestyle='dashed',label='tE_upper_limit')
plt.xlabel('tobs')
plt.ylabel('N')
plt.legend()
rec_list_1d[col_indices[3]], dec_list_1d[row_indices[3]]
print(Time(2016.6, format='decimalyear').jd -2450000)

In [ ]:
#MICROLENSING
ra, dec, pmra, pmdec, parallax, UWE = [], [], [], [], [], []
u_0, theta_E, t_0, t_E, pi_EN, pi_EE = [], [], [], [], [], []

ra_fit,dec_fit,ra_fit_nol,dec_fit_nol=[],[],[],[]
pmra_fit, pmdec_fit, parallax_fit,pmra_fit_err, pmdec_fit_err, parallax_fit_err = [], [], [],[], [], []
pmra_fit_nol, pmdec_fit_nol, parallax_fit_nol,UWE_nol  = [], [], [], []
pmra_fit_nol_err, pmdec_fit_nol_err, parallax_fit_nol_err  = [], [], []

Nsource=5000
no_noise=False # To generate mock observations without noise and determine if different models 
#are used when generating tracks and performing 11-parameter fit
noise_and_no_noise=False
add2parquet=True # for adding extra values (ldracs, lddecs in mag_diff) to .parquet files
fix_all_params = False

for i in range(1):
    print(range(1))
for i in range(Nsource):
  pmd = np.random.uniform(-40, 40)
  pma = np.random.uniform(-40, 40)
  pi = np.random.uniform(1e-5, 2) # not from 0 because distance is 1/pi
  #pmd=0.
  #pma=0.
  #pi=0.1
  ra.append(RA)
  dec.append(DEC)
  pmra.append(pma)
  pmdec.append(pmd)
  parallax.append(pi)

  SOURCE_ID: int = generate_random_digits(19) # why is this necessary?

  piEN=np.random.uniform(-1, 1)
  piEE=np.random.uniform(-1, 1)
  u0=np.random.uniform(-5, 5) #change here
    
  #tE=np.random.uniform(1, 365) # from 0-1year, 2 years might be too long
  tE=np.random.uniform(30, 180) 
    
  #t0=np.random.uniform(6910, 7830) # from 2014.7-2017.2 - DR3
  t0=np.random.uniform(7133, 7608.1) # from 2015.3-2016.6
  #t0=7265 # 2015.65

  thetaE = np.random.uniform(0.1, 10)
  #thetaE = np.random.uniform(1, 5)
  #thetaE=2
    
  if fix_all_params:
    thetaE, u0, tE, t0, piEN, piEE=5, 1, 300, 6500, 0.1, -0.2
    
  params = astromet.params()
  params.epoch=REFERENCE_EPOCH

  params = astromet.define_lens(params, RA, DEC,
                              u0=u0,
                              t0=t0,
                              tE=tE,
                              piEN=piEN,
                              piEE=piEE,
                              m0=mean_g_mag,
                              fbl=1,
                              pmrac_source=pmra[i],
                              pmdec_source=pmdec[i],
                              d_source=1/parallax[i],
                              thetaE=thetaE)
  #no lensing
  params_nol = astromet.params()
  params_nol.epoch=REFERENCE_EPOCH
  params_nol.ra = RA
  params_nol.dec = DEC
  params_nol.pmrac = pmra[i]
  params_nol.pmdec = pmdec[i]
  params_nol.parallax = parallax[i]

  ldracs, lddecs, mag_diff = astromet.track(ts, params)
  ldracs_nol, lddecs_nol = astromet.track(ts, params_nol)
  nmeasure=9
  magnitude_measurements = np.repeat(mag_diff, nmeasure)+mean_g_mag
  if no_noise:
      ast_err,ast_err_nol=0.,0.
  else:
      #ast_err,ast_err_nol = astromet.sigma_ast(magnitude_measurements),astromet.sigma_ast(mean_g_mag)
      ast_err,ast_err_nol = 0.1*astromet.sigma_ast(magnitude_measurements),0.1*astromet.sigma_ast(mean_g_mag) #### for reduced noise #####

  t_obs,x_obs,phi_obs,rac_obs,dec_obs = astromet.mock_obs(ts, phis, ldracs, lddecs, err=ast_err, nmeasure=nmeasure) 
  t_obs_nol, x_obs_nol, phi_obs_nol, rac_obs_nol, dec_obs_nol = astromet.mock_obs(ts, phis, ldracs_nol, lddecs_nol, err=ast_err_nol, nmeasure=nmeasure)

  #no_noise: Not possible to set noise=0. UWE is calculated as reqsults['uwe']= np.sqrt(np.sum(R**2 / xerr**2)/(np.sum(weights>0.2)-nparam)) which gives nan if ast_err=0.
  results = astromet.fit(t_obs, x_obs, phi_obs, ast_err, RA, DEC)
  results_nol = astromet.fit(t_obs_nol, x_obs_nol, phi_obs_nol, ast_err_nol, RA, DEC)
  if no_noise:
     if add2parquet:
         #### repeat ldracs, lddecs in mag_diff nmeasure-times #########
         ### for single source there is no mag_diff, so set it to 0 ####
         mock_lpc = to_mock_lpc_add(
            SOURCE_ID,
            mean_g_mag,
            REFERENCE_EPOCH,
            results,
            x_obs, x_obs_nol,
            np.zeros(len(x_obs)),
            phi_obs,
            rac_obs,dec_obs,
            np.repeat(ldracs, 9),np.repeat(lddecs, 9),np.repeat(mag_diff, 9))
     else:
         mock_lpc = to_mock_lpc(
            SOURCE_ID,
            mean_g_mag,
            REFERENCE_EPOCH,
            results,
            x_obs, x_obs_nol,
            np.zeros(len(x_obs)),
            phi_obs,
            rac_obs, dec_obs)
  else:
      if add2parquet:
          mock_lpc = to_mock_lpc_add(
            SOURCE_ID,
            mean_g_mag,
            REFERENCE_EPOCH,
            results,
            x_obs, x_obs_nol,
            ast_err,
            phi_obs,
            rac_obs, dec_obs,
            np.repeat(ldracs, 9),np.repeat(lddecs, 9),np.repeat(mag_diff, 9))
      else:
          mock_lpc = to_mock_lpc(
            SOURCE_ID,
            mean_g_mag,
            REFERENCE_EPOCH,
            results,
            x_obs, x_obs_nol,
            ast_err,
            phi_obs,
            rac_obs, dec_obs)
    
  u_0.append(u0)
  t_E.append(tE)
  pi_EN.append(piEN)
  pi_EE.append(piEE)
  theta_E.append(thetaE)
  mock_lpc.to_parquet("mock_lpc_" + str(i+1) + ".parquet")
  UWE.append(results['uwe'])
  t_0.append(t0)
  ra_fit.append(results['ra_ref']+results['drac']*mas/np.cos(results['dec_ref']))
  dec_fit.append(results['dec_ref']+results['ddec']*mas)
  #ra_fit.append(results['drac']+RA)
  #dec_fit.append(results['ddec']+DEC)
  pmra_fit.append(results['pmrac'])
  pmdec_fit.append(results['pmdec'])
  parallax_fit.append(results['parallax'])
  #pmra_fit_err.append(results['pmrac_error'])
  #pmdec_fit_err.append(results['pmdec_error'])
  #parallax_fit_err.append(results['parallax_error'])

  ra_fit_nol.append(results['ra_ref']+results['drac']*mas/np.cos(results['dec_ref']))
  dec_fit_nol.append(results['dec_ref']+results['ddec']*mas)
  #ra_fit_nol.append(results_nol['drac']+RA)
  #dec_fit_nol.append(results_nol['ddec']+DEC)
  pmra_fit_nol.append(results_nol['pmrac'])
  pmdec_fit_nol.append(results_nol['pmdec'])
  parallax_fit_nol.append(results_nol['parallax'])
  #pmra_fit_nol_err.append(results_nol['pmrac_error'])
  #pmdec_fit_nol_err.append(results_nol['pmdec_error'])
  #parallax_fit_nol_err.append(results_nol['parallax_error'])
  UWE_nol.append(results_nol['uwe'])
UWE_microlensing=UWE

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(15, 6), subplot_kw=dict(box_aspect=0.5))

axs[0].scatter(pmra_fit_nol,pmra_fit,s=1)
axs[0].set_title('pmra [mas/yr]')
axs[0].xaxis.set_label_text('fit_nol')
axs[0].yaxis.set_label_text('fit_ml')
axs[1].scatter(pmdec_fit_nol,pmdec_fit,s=1)
axs[1].set_title('pmdec [mas/yr]')
axs[1].xaxis.set_label_text('fit_nol')
axs[1].yaxis.set_label_text('fit_ml')
axs[2].scatter(parallax_fit_nol,parallax_fit,s=1)
axs[2].set_title('parallax [mas]')
axs[2].xaxis.set_label_text('fit_nol')
axs[2].yaxis.set_label_text('fit_ml')

fig.suptitle("Microlensing", fontsize=18,y=0.75)
fig.savefig(figure_dir + 'params_fit_ml_vs_fit_nol.png', dpi=150, bbox_inches='tight')

fig, axs = plt.subplots(1, 2, figsize=(15, 6), subplot_kw=dict(box_aspect=0.5))
axs[0].scatter(ra,ra_fit,s=1)
axs[0].set_title('ra [deg]')
axs[0].xaxis.set_label_text('true_value')
axs[0].yaxis.set_label_text('fitted_value')
axs[1].scatter(dec,dec_fit,s=1)
axs[1].set_title('dec [deg]')
axs[1].xaxis.set_label_text('true_value')
axs[1].yaxis.set_label_text('fitted_value')


fig.suptitle("Microlensing", fontsize=18,y=0.8)
fig.savefig(figure_dir + 'params_fit_ml_vs_fit_nol.png', dpi=150, bbox_inches='tight')

fig, axs = plt.subplots(1, 3, figsize=(15, 6), subplot_kw=dict(box_aspect=0.5))
axs[0].scatter(pmra,pmra_fit,s=1)
axs[0].set_title('pmra [mas/yr]')
axs[0].xaxis.set_label_text('true_value')
axs[0].yaxis.set_label_text('fitted_value')
axs[1].scatter(pmdec,pmdec_fit,s=1)
axs[1].set_title('pmdec [mas/yr]')
axs[1].xaxis.set_label_text('true_value')
axs[1].yaxis.set_label_text('fitted_value')
axs[2].scatter(parallax,parallax_fit,s=1)
axs[2].set_title('parallax [mas]')
axs[2].xaxis.set_label_text('true_value')
axs[2].yaxis.set_label_text('fitted_value')

fig.suptitle("Microlensing", fontsize=18,y=0.75)
fig.savefig(figure_dir + 'params_fit_vs_true_microlensing.png', dpi=150, bbox_inches='tight')


In [ ]:
fig, axs = plt.subplots(4, 3, figsize=(15, 12), subplot_kw=dict(box_aspect=0.5))
font_size=16

axs[0, 0].hist(ra,bins=100,color='red')
axs[0, 0].set_xlabel('ra_source [deg]', fontsize=font_size)
axs[0, 0].tick_params(labelsize=font_size)
axs[0, 1].hist(dec,bins=100,color='red')
axs[0, 1].set_xlabel('dec_source [deg]', fontsize=font_size)
axs[0, 1].tick_params(labelsize=font_size)
axs[0, 2].hist(pmra,bins=500,color='red')
axs[0, 2].set_xlabel('pmrac_source [mas/yr]', fontsize=font_size)
axs[0, 2].tick_params(labelsize=font_size)
axs[1, 0].hist(pmdec,bins=500,color='red')
axs[1, 0].set_xlabel('pmdec_source [mas/yr]', fontsize=font_size)
axs[1, 0].tick_params(labelsize=font_size)
axs[1, 1].hist(parallax,bins=500,color='red')
axs[1, 1].set_xlabel('parallax [mas]', fontsize=font_size)
axs[1, 1].tick_params(labelsize=font_size)
axs[1, 2].hist(UWE,bins=500,color='red')
axs[1, 2].set_xlabel('UWE', fontsize=font_size)
axs[1, 2].tick_params(labelsize=font_size)
axs[2, 0].hist(u_0,bins=500,color='red')
axs[2, 0].set_xlabel('u0 [mas]', fontsize=font_size)
axs[2, 0].tick_params(labelsize=font_size)
axs[2, 1].hist(theta_E,bins=500,color='red')
axs[2, 1].set_xlabel('thetaE [mas]', fontsize=font_size)
axs[2, 1].tick_params(labelsize=font_size)
axs[2, 2].hist(t_E,bins=500,color='red')
axs[2, 2].set_xlabel('tE [days]', fontsize=font_size)
axs[2, 2].tick_params(labelsize=font_size)
axs[3, 0].hist(t_0,bins=500,color='red')
axs[3, 0].set_xlabel('t0 [reduced JD]', fontsize=font_size)
axs[3, 0].tick_params(labelsize=font_size)
axs[3, 1].hist(pi_EE,bins=500,color='red')
axs[3, 1].set_xlabel('piEE [mas]', fontsize=font_size)
axs[3, 1].tick_params(labelsize=font_size)
axs[3, 2].hist(pi_EN,bins=500,color='red')
axs[3, 2].set_xlabel('piEN [mas]', fontsize=font_size)
axs[3, 2].tick_params(labelsize=font_size)
#for ax in axs.flat:
#    ax.set(xlabel='x-label', ylabel='y-label')
fig.subplots_adjust(hspace=0.5) #adjust vertical space
fig.subplots_adjust(wspace=0.3) #adjust width

#fig.suptitle("Microlensing", fontsize=18, y=0.92)
fig.savefig(figure_dir + 'histogram_microlensing.png', dpi=150, bbox_inches='tight')

In [ ]:
n_rows = len(u_0)
d = {'u_0': u_0,
     'theta_E': theta_E,
     't_E': t_E,
     't_0': t_0,
     'pi_EE': pi_EE,
     'pi_EN': pi_EN}
microlensing_params = pd.DataFrame(data = d, index = np.arange(1, Nsource+1))

In [ ]:
microlensing_params.to_csv('microlensing_params.csv', index=False)

In [ ]:
d = {'ra': ra,
  'dec': dec,
 'pmra': pmra,
 'pmdec': pmdec,
 'parallax': parallax,
 'UWE': UWE}
params = pd.DataFrame(data = d, index = np.arange(1,Nsource+1))
params.to_csv('params.csv', index=False)

In [ ]:
# Fitted values
d_fit = {'ra': ra_fit,
  'dec': dec_fit,
 'pmra': pmra_fit,
 'pmdec': pmdec_fit,
 'parallax': parallax_fit,
 'UWE': UWE}
params_fit = pd.DataFrame(data = d_fit, index = np.arange(1,Nsource+1))
params_fit.to_csv('params_fit.csv', index=False)


In [ ]:
! zip -r $game_dir/microlensing_only.zip *

# Comparing UWE of different sources

In [ ]:
bin_list=np.linspace(0,2,500)
fig = plt.figure(figsize=(5, 5))

#plt.hist(UWE_sing_source,bins=bin_list,color='blue',alpha=0.5,label='single source')
plt.hist(UWE_microlensing,bins=bin_list,color='red',alpha=0.5,label='microlensing')

plt.xlabel('UWE')
plt.ylabel('Counts')
plt.legend(loc='upper right')
plt.xlim((0.,2))
plt.title('UWE (N='+str(Nsource)+' for each type)')
fig.savefig(figure_dir + 'histogram_all_sources.png', dpi=150, bbox_inches='tight')